Import dependencies

In [1]:
import folium

###### Build map centered at alabama

In [2]:
gun_map = folium.Map(location=[32.3182, -86.9023], tiles='Stamen Toner', zoom_start=7)

Check up on our data

In [3]:
import pandas as pd

gun_df = pd.read_csv('guns_subset.csv')
gun_df.head()

FileNotFoundError: File b'guns_subset.csv' does not exist

In [4]:
print('Number of gun related incidents in Alabama : ', len(gun_df))

NameError: name 'gun_df' is not defined

There is some missing information that I would like to have to create my map, I need to fill in the latitude & longitude values, I'm thinking that I can use n_injured/n_killed to create map subsets.  'Unnamed: 0', 'X' are indexing artifacts from building the data file from the original dataset, I can safely remove those to clean up my workspace.

In [5]:
gun_df = gun_df.drop(['Unnamed: 0', 'X'], axis=1)
gun_df.head()

,address,city_or_county,date,incident_url,latitude,longitude,n_injured,n_killed
0,1300 block of Alabama Ave,Selma,2014-09-01,http://www.gunviolencearchive.org/incident/186796,NaN,NaN,0,0
1,5900 Messer Airport Hwy,Birmingham,2014-09-01,http://www.gunviolencearchive.org/incident/192477,NaN,NaN,0,0
2,3100 block of 20th Street,Tuscaloosa,2014-09-01,http://www.gunviolencearchive.org/incident/186638,NaN,NaN,1,0
3,502 Barnett Boulevard,Tallassee,2014-09-02,http://www.gunviolencearchive.org/incident/186827,NaN,NaN,0,0
4,1500 block of 53rd St,Birmingham (Ensley),2014-09-02,http://www.gunviolencearchive.org/incident/186723,NaN,NaN,0,1


That looks a bit cleaner, lets see if we can get some GPS coordinates for marking

In [6]:
from geopy.geocoders import Nominatim

geolocater = Nominatim()
address = gun_df.address[2] + ' ' + gun_df.city_or_county[2] + ', Alabama'
print(address)
print(geolocater.geocode(address))

print('That did not work... lets experiment')
print(geolocater.geocode('3100 20th Street  Tuscaloosa, Alabama'))

3100 block of 20th Street  Tuscaloosa, Alabama
None
That did not work... lets experiment
3100, 20th Street, West End, Tuscaloosa, Tuscaloosa County, Alabama, 35401, United States of America


Seems that the geolocation library doesn't know what to do with words groups like 'of', 'block of', etc.  I need to build columns with street numbers, street names, etc.

In [23]:
import usaddress as usa
address = usa.tag(gun_df.address[2] + ' ' + gun_df.city_or_county[2])
print(address)

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U21') dtype('<U21') dtype('<U21')

That didn't work like I had hoped, I'm going to just have to cut out those garbage words manually.  Lets get a better Idea of what I'm working with

In [25]:
gun_df.address[:30]

0                    1300 block of Alabama Ave
1                      5900 Messer Airport Hwy
2                   3100 block of 20th Street 
3                        502 Barnett Boulevard
4                        1500 block of 53rd St
5               100 block of W. Delano Avenue 
6               200 block of 4th Terrace North
7                                   4920 US-78
8           4000 block of Hunters Ridge Drive 
9                5100 block of Carmichael Road
10                   Brindlee Mountain Parkway
11                Athey Road at Skipper Drive 
12    1600 block of Jefferson Avenue Southwest
13             1900 block of Center Point Road
14           Harden Avenue and Governors Drive
15                                Jordan Lane 
16                       901 East Meighan Blvd
17                          Lake Forest Circle
18             3500 block of Griffintown Road 
19              3000 block of Meridian Street 
20                   3400 block of Conley Road
21           

Looks like the main offender is:

'block of'

That looks way better!

I dont have high hopes that this solved all cases, but I think it took care of most occurences.  We'll know for sure later.

In [26]:
gun_df['address'] = gun_df['address'].replace('block of', '')
print(gun_df.head())
# location = geolocater.geocode(gun_df['address'][2] + ' ' + gun_df['city_or_county'][2] + ' AL')
# print(location)
# print(location.latitude, location.longitude)

   Unnamed: 0    X                     address       city_or_county  \
0           1   19   1300 block of Alabama Ave                Selma   
1           2   44     5900 Messer Airport Hwy           Birmingham   
2           3  152  3100 block of 20th Street            Tuscaloosa   
3           4  304       502 Barnett Boulevard            Tallassee   
4           5  305       1500 block of 53rd St  Birmingham (Ensley)   

         date                                       incident_url  latitude  \
0  2014-09-01  http://www.gunviolencearchive.org/incident/186796       NaN   
1  2014-09-01  http://www.gunviolencearchive.org/incident/192477       NaN   
2  2014-09-01  http://www.gunviolencearchive.org/incident/186638       NaN   
3  2014-09-02  http://www.gunviolencearchive.org/incident/186827       NaN   
4  2014-09-02  http://www.gunviolencearchive.org/incident/186723       NaN   

   longitude  n_injured  n_killed  
0        NaN          0         0  
1        NaN          0         

In [10]:
latitude = list()
longitude = list()
for i, a in enumerate(gun_df['address']):
    if(i % 10 == 0):
        print('Percent complete : %', i*100/len(gun_df['address']))
    location = geolocater.geocode(str(a) + ' ' + str(gun_df['city_or_county'][i]) + ' AL')
    try:
#     if (location is not None):
        
        gun_df['latitude'][i] = location.latitude
        gun_df['longitude'][i] = location.longitude
    except:
        print('Had trouble with index :', i)



Percent complete : % 0.0
Had trouble with index : 0


/Users/austingreen/.pyenv/versions/jeff_co/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/austingreen/.pyenv/versions/jeff_co/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Had trouble with index : 2
Had trouble with index : 4
Had trouble with index : 5
Had trouble with index : 6
Had trouble with index : 7
Had trouble with index : 8
Had trouble with index : 9
Percent complete : % 0.09062896501721951
Had trouble with index : 10
Had trouble with index : 11
Had trouble with index : 12
Had trouble with index : 13
Had trouble with index : 14
Had trouble with index : 15


KeyboardInterrupt: 

That took a while.  I'm going to write to a CSV so we never have to run that cell again.

In [ ]:
gun_df

In [14]:
latitude = list()
longitude = list()
print('hi')
for i, a in enumerate(gun_df['address']):
    print(str(a) + ' ' + str(gun_df['city_or_county'][i]) + ' AL')
    

hi
1300 block of Alabama Ave Selma AL
5900 Messer Airport Hwy Birmingham AL
3100 block of 20th Street  Tuscaloosa AL
502 Barnett Boulevard Tallassee AL
1500 block of 53rd St Birmingham (Ensley) AL
100 block of W. Delano Avenue  Montgomery AL
200 block of 4th Terrace North Birmingham AL
4920 US-78 Oxford AL
4000 block of Hunters Ridge Drive  Huntsville AL
5100 block of Carmichael Road Montgomery AL
Brindlee Mountain Parkway Arab AL
Athey Road at Skipper Drive  Mobile AL
1600 block of Jefferson Avenue Southwest Birmingham AL
1900 block of Center Point Road Birmingham AL
Harden Avenue and Governors Drive Huntsville AL
Jordan Lane  Huntsville AL
901 East Meighan Blvd Gadsden AL
Lake Forest Circle Hoover AL
3500 block of Griffintown Road  Bibb County AL
3000 block of Meridian Street  Huntsville AL
3400 block of Conley Road Hoover AL
100 block of Broad Street  Alexander City  AL
300 block of Beta Street Birmingham AL
Washington St Greensboro AL
300 block of Eighth Street Pleasant Grove AL
31

Azalea Road and Pleasant Valley Road Mobile AL
nan Notasulga AL
600 block of Montana Street Dothan AL
4405 Brewbaker Dr Montgomery AL
1434 Thomason Ave Birmingham (Tarrant) AL
4808 University Dr NW Huntsville AL
205 6th Ave Decatur AL
W.F. Gardner Drive and U.S. 72 Tuscumbia AL
5800 block of Underwood Ave Birmingham AL
4336 Newton St Birmingham AL
300 block of Highway 437 Sterrett AL
100 block of Easy Street Opelika AL
1400 block of College Street Flomaton AL
Glade Road East and Weaver Road Anniston AL
2916 Gallant Drive Birmingham AL
4100 Messer Airport Hwy Birmingham AL
200 block of Sherwin Street Enterprise AL
1100 block of U.S. Highway 231 Troy AL
6000 block of 1st Ave N Birmingham AL
2200 block of Northland Avenue Southwest Birmingham AL
2801 9th Ave N Bessemer AL
2500 block of Clarendon Avenue Bessemer AL
2100 block of 31st Street Birmingham AL
North Carter Road Crenshaw (county) AL
West Street Bessemer (Brighton) AL
Industrial Drive Dothan AL
4000 block of Admore Drive Montgomer

3810 7th Ave Bessemer (Brighton) AL
1300 block of Cloverdale Ave Morgan (county) AL
512 E Columbus St Dadeville AL
800 block of N Gay St Auburn AL
nan Ozark AL
US-78 Anniston AL
Jefferson Avenue SW and Mims Street Birmingham AL
Lee Road 401 and Lee Road 170 Lee (county) AL
Lee Road 401 and Lee Road 170 Salem AL
2500 Barney Terrace NW Huntsville AL
Poser Road Foley AL
Montclair Road Birmingham AL
Smith Road Limestone (county) AL
700 block of 81st Place South Birmingham AL
Pleasant Valley Road Mobile AL
Helen Keller Blvd. Tuscaloosa AL
Denna Road Valley AL
1700 block of 26th Avenue North Birmingham AL
100 block of Arabian Drive Madison AL
nan Heflin AL
6800 block of McFarland Boulevard Tuscaloosa (county) AL
1600 John Wesley Way NW Huntsville AL
500 block of 41st St N Birmingham AL
U.S. Highway 78 Lincoln AL
nan Birmingham AL
South Craft Highway and Chieftain Way Mobile (Chickasaw) AL
3rd Ave North and 6th Place North Birmingham AL
2400 block of 9th Court South Birmingham AL
nan Limeston

4300 block of 12th Avenue Birmingham AL
St. James and Seay Streets Montgomery AL
1st Ave North Birmingham AL
2100 block of 31st Avenue North Birmingham AL
Sherry Dr Mobile AL
10th Ave. Huntsville AL
4815 Reichhold Road Tuscaloosa AL
600 block of Carraway Boulevard Birmingham AL
Bessemer Super Highway Birmingham (Midfield) AL
Forestdale Boulevard Jefferson (county) AL
North Crown Street Florence AL
West Montgomery Road Tuskegee AL
3400 block of Gilmer Court Montgomery AL
Sea Warrior Road Choctaw (county) AL
121 Marella Court Dothan AL
Wilson Boulevard and Tim Street Russellville AL
200 block of Saturn Lane Jefferson (county) AL
6500 block of Todd Acres Drive Theodore AL
Jim Platt Road Citronelle AL
nan Jefferson (county) AL
24819 US-31 Jemison AL
I-85 Montgomery AL
1007 Danville Road Southwest Decatur AL
Academy Court Bessemer AL
1121 Huffman Road Birmingham AL
2349 Forestdale Boulevard Birmingham AL
14th Street Phenix City AL
Richard Arrington Boulevard Birmingham AL
Green Valley Road 

14096 Memorial Parkway Southwest Huntsville AL
1300 Kent Drive Auburn AL
McFarland Boulevard  Tuscaloosa AL
12th Avenue Tuscaloosa AL
2100 block of Wildwood Drive Millbrook AL
5500 14th Ave Birmingham AL
1200 block of Church Street Huntsville AL
2200 block of North Memorial Parkway Huntsville AL
3300 Charleston Avenue Huntsville AL
Cullman Circle Lisman AL
Highway 11 Steele AL
Avenue D and 31st Street Birmingham AL
Church Street Huntsville AL
Jones Street Prattville AL
1500 block of Warrior Road Birmingham AL
700 block of 24th Avenue Northwest Birmingham AL
1700 block of St. Philips St.  Selma AL
Centre Road Gadsden AL
Net Street and Twine Street Blue Mountain AL
100 block of Spirit Drive Toney AL
1001 Little Sorrel Dr Calera AL
Sledge Road Athens AL
Pratt Avenue Huntsville AL
4901 Zeigler Blvd Mobile AL
1000 West Harlem Avenue  Hueytown AL
2900 block of Polar Avenue Huntsville AL
5200 Gunn Road Mobile AL
Oporto Madrid Boulevard Birmingham AL
4751 Governor's House Dr. Huntsville AL
Eas

Jew Hollow Road Mount Olive AL
South Crown Street Florence AL
800 Airport Dr Dothan AL
300 block of Cherry Avenue Opelika AL
180 block of 2nd Street Thomas  Birmingham AL
Park Street Troy AL
 248 1st Ave Sw Birmingham AL
1477 Gadsden Hwy Birmingham AL
Logan Road  Clanton AL
4100 block of 5th Court North  Birmingham AL
Upper Vienna Road Aliceville AL
410 Chestnut St Gadsden AL
226 49th Ave E Tuscaloosa AL
nan Pickens County (county) AL
910 Martin Luther King Blvd S Union Springs  AL
1207 Old Trinity Road Trinity AL
1500 block of Bessemer Road  Birmingham AL
1100 block of Alabama Avenue Selma AL
1000 block of Houston Street  Mobile AL
3401 Holmes Ave NW Huntsville AL
22000 of Fain Road Elkmont AL
Highway 59  Bay Minette  AL
Highway 98 Magnolia Springs AL
Main Avenue and Alabama 21 Sylacauga AL
2700 block of 35th Avenue North Birmingham AL
6800 block of Division Avenue  Birmingham AL
3400 block of Margaret Ann Drive Montgomery AL
1800 block of Green Street Selma AL
1500 14th Street Phenix

2 Pirate Dr Fairhope AL
1st Street West Birmingham AL
nan Vernon AL
100 block of Clover Ln Montgomery AL
1300 block of 45th St Birmingham (Ensley) AL
Spring Creek Circle Mobile AL
6th Ave and 4th St Birmingham AL
4000 block of Smiley Circle Montgomery AL
1000 block of 46th St Birmingham (Ensley) AL
First Avenue and Main Street Mobile AL
nan Calhoun (county) AL
Highway 31 Birmingham (Hoover) AL
1115 Ave F Birmingham AL
4000 block of Oliver Dr Millbrook AL
100 block of Chester Avenue Opelika AL
1627 Opelika Rd Auburn AL
1910 St Stephens Road Mobile AL
King Street Ozark AL
3725 27th St N Birmingham AL
1200 block of Amy Ct Auburn AL
2300 block of South Park Dr Birmingham AL
2700 block of 8th Ave S Bessemer AL
4200 block of 21st Street Tuscaloosa AL
nan Cullman AL
nan Mobile (Prichard) AL
5954 Sperry Rd Theodore AL
1242 Beltline Rd SW Decatur AL
6550 Zeigler Blvd Mobile AL
3900 block of Woodley Road Montgomery AL
1430 Fred L Shuttlesworth Dr Birmingham AL
6th St Birmingham AL
1500 block of 

1700 Block of Jefferson Ave SW Birmingham AL
2200 block of Harrison St SE Decatur AL
268 Trinity Ln Brewton AL
212 Oakmont St Bessemer (Brighton) AL
1600 block of Faure Dr E Mobile AL
Childress Ave Bayou La Batre AL
26541 Schoen Rd Elberta AL
305 S Perry St Montgomery AL
515 S Eufaula Ave Eufaula AL
nan Birmingham AL
nan Birmingham AL
1485 Jack Springs Rd Atmore AL
2800 block of Seventh Avenue South Birmingham AL
South Randolph Avenue Eufaula AL
4100 block of Oak Ridge Cir Bessemer (Brighton) AL
7137 Old Jasper Hwy Dora AL
nan Pine Hill AL
2700 block of Matthew Ln Selma AL
1500 block of Washington St Selma AL
112 US 82 Prattville AL
811 Walnut St Dothan AL
Mustang Dr Dothan AL
705 N Lena St Dothan AL
N Cherokee St and Westmoreland St Florence AL
1200 block of Alabama Ave Selma AL
2800 block of Fairbanks Street Huntsville AL
5000 block of 1st Ave N Birmingham AL
Montgomery Drive Ashville AL
6800 block of Kimberly Avenue Birmingham AL
Carnegie Dr Fairfield AL
100 block of Sibert Street M

1200 Columbus Parkway Opelika AL
6969 Gadsden Hwy Trussville AL
137 Patrick Street Prattville AL
2166 Wagner St Mobile AL
Meaher St Mobile (Prichard) AL
5600 block of Carmichael Road Montgomery AL
30th Pl Tuscaloosa AL
171 Starmont Road Gallion AL
1900 block of Toomer Street Opelika AL
Wheeler Avenue and Fiber Street Huntsville AL
3425 St Stephens Rd Mobile (Prichard) AL
651 Azalea Road Mobile AL
7800 block of 4th Avenue South Birmingham AL
500 block of East South Boulevard Montgomery AL
3300 block of Fountain Lane Montgomery AL
3000 block of 19th Street Tuscaloosa AL
1700 block of 33rd Avenue Tuscaloosa AL
5800 block of Eagle Circle Montgomery AL
1000 block of Villa Rica Court Birmingham AL
6458 Tanner Street Mobile AL
nan Alberta AL
5609 US 80 East Lowndesboro AL
Ross Clark Circle Dothan AL
28th Street Birmingham AL
3708 McVay St SW Huntsville AL
505 Bibb Street Bay Minette AL
nan Albertville AL
3100 block of Lower Wetumpka Road Montgomery AL
14595 U.S. 231-431 Hazel Green AL
1000 bl

1901 Sparkman Dr NW Huntsville AL
2400 block of Elvester Rd Warrior AL
200 block of Robin St Hamilton AL
1st Ave Selma AL
Phyllis Dr Gadsden AL
Barbara Dr Mobile AL
8700 block of Three Notch Road Mobile AL
Falco Road Andalusia AL
AL-91 and County Road 747 Hanceville AL
County Road 24 Crossville AL
100 block of 12th Avenue North Birmingham AL
Schillinger Road Mobile AL
nan Tuskegee AL
5500 block of Gantry Drive Montgomery AL
2000 block of Northland Ave Birmingham AL
704 S US Highway 31 Bay Minette AL
300 block of Autumn Ln Madison AL
1076 3rd St Florala AL
3300 block of 36th Street Tuscaloosa AL
US 82 and Countryside Road Centreville AL
Blackmon Road Brantley AL
8000 block of Rugby Drive Birmingham AL
1602 Brookridge Rd SW Decatur AL
210 Hickory St Bay Minette AL
1715 Martin Luther King Jr Blvd Tuscaloosa AL
Chandler St Gadsden AL
600 block of Dogwood Ln Tuscaloosa AL
nan Bay Minette AL
US-98 and Keller Rd Fairhope AL
1700 block of Alameda Ct Birmingham AL
2200 block of Lynnchester Cir 